In [ ]:
from openai import OpenAI
import time


client = OpenAI(api_key = "your_api_key")

# 加载 JSON 数据
with open("interesting_pairs.json", "r") as f:
    data = json.load(f)

# 转换为 DataFrame 并筛选 overlap_count >= 3
df = pd.DataFrame(data)
print(f"Read {len(df)} data.")
df_filtered = df[df["overlap_count"] >= 3].copy()
print("overlap_count >= 3 data #:", len(df_filtered))

# 初始化结果列
df_filtered["gpt_score"] = None
df_filtered["gpt_reason"] = None

# 模板提示词
def create_prompt(pair1, pair2):
    return f"""You are an expert in biomedical informatics. Please evaluate the biological or clinical relevance between the following two traits:

- Trait A: {pair1}
- Trait B: {pair2}

Each trait has a format like 'Disease--Condition'. 'Condition' might be 'None', which means 'regardless of any condition'.

Based on your knowledge, assign a score and reasoning in the following JSON format:
{{
  "score": 3, // if there is clear evidence or published research confirming their relationship
  "score": 2, // if there is a possible connection, but no strong evidence or confirmation
  "score": 1, // if you believe they are likely unrelated
  "reason": "Your explanation here, ideally with references or reasoning."
}}

Only generate the json, and do not generate anything else."""

# 遍历每条记录并调用 GPT-4o API
for i, row in df_filtered.iterrows():
    prompt = create_prompt(row["pair1"], row["pair2"])
    
    try:
        chat_response = client.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": prompt}],
            temperature=0
        )

        reply = chat_response.choices[0].message.content
        # 提取第一个 '{' 和最后一个 '}' 之间的 JSON 部分
        start = reply.find("{")
        end = reply.rfind("}") + 1
        json_str = reply[start:end]
        
        # 尝试解析 JSON
        try:
            result = json.loads(json_str)
        except json.JSONDecodeError as json_err:
            print(f"❌ JSON parse failed at row {i}: {json_err}")
            df_filtered.at[i, "gpt_score"] = "json_error"
            df_filtered.at[i, "gpt_reason"] = reply  # 原始内容保留
            continue
        
        # 存入 DataFrame
        df_filtered.at[i, "gpt_score"] = result.get("score")
        df_filtered.at[i, "gpt_reason"] = result.get("reason")
        
        print(f"✅ Done [{i}]: {row['pair1']} -- {row['pair2']} | Score: {result.get('score')}")
        time.sleep(1.5)  # 避免过快请求触发速率限制

    except Exception as e:
        print(f"❌ Failed [{i}] — {row['pair1']} vs {row['pair2']}: {e}")
        print(reply)
        df_filtered.at[i, "gpt_score"] = "error"
        df_filtered.at[i, "gpt_reason"] = str(e)
        time.sleep(5)  # 错误时等待更久

# 保存最终结果
df_filtered.to_csv("gpt4o_evaluated_pairs.csv", index=False)
print("saved to gpt4o_evaluated_pairs.csv")


Read 1214 data.
overlap_count >= 3 data #: 462
✅ Done [2]: Acute_Myeloid_Leukemia--Hypertension -- Ankylosing_Spondylitis--None | Score: 2
✅ Done [7]: Acute_Myeloid_Leukemia--Hypertension -- Celiac_Disease--Ankylosing_Spondylitis | Score: 2
✅ Done [10]: Acute_Myeloid_Leukemia--Hypertension -- Celiac_Disease--Obesity | Score: 2
✅ Done [11]: Acute_Myeloid_Leukemia--Hypertension -- Celiac_Disease--Psoriasis | Score: 2
✅ Done [12]: Acute_Myeloid_Leukemia--Hypertension -- Coronary_artery_disease--None | Score: 2
✅ Done [13]: Acute_Myeloid_Leukemia--Hypertension -- Cystic_Fibrosis--Gender | Score: None
❌ JSON parse failed at row 16: Extra data: line 5 column 1 (char 612)
✅ Done [19]: Acute_Myeloid_Leukemia--Hypertension -- Insomnia--None | Score: 2
✅ Done [21]: Acute_Myeloid_Leukemia--Hypertension -- Osteoarthritis--Hypertension | Score: 2
✅ Done [22]: Acute_Myeloid_Leukemia--Hypertension -- Osteoarthritis--Liver_cirrhosis | Score: 2
✅ Done [23]: Acute_Myeloid_Leukemia--Hypertension -- Osteo

# 初步评估

得分3分的几种疾病组合,虽然数量不多(9/373),但是说明我们发现的东西是convincing的. 

得分为1分的组合有很多(126/373),属于非常规的correlation,这些可以作为推荐研究drug repurposing的方向.
